In [1]:
import os
import re
import numpy as np
import pandas as pd
import warnings

import seaborn as sns
import matplotlib.pyplot as plt

import nltk
from bs4 import BeautifulSoup

from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

stopwords = nltk.corpus.stopwords.words("english")
warnings.filterwarnings("ignore")

In [2]:
csv_file = "data\IMDB_dataset.csv"
df = pd.read_csv(csv_file)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
### utilities
def remove_special_characters(text):
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()
    text = re.sub('\[[^]]*\]', '', text)
    text = re.sub('[^a-zA-z0-9\s\.,]','',text)
    return text

def remove_stopwords(text):
    text = text.lower()
    words = text.split(" ")
    filtered_words = [i for i in words if i not in stopwords]
    filtered_words = ' '.join(filtered_words)
    return filtered_words
    

In [4]:
## viewing a sample review before applying data transformation
df['review'].values[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [5]:
df['review'] = df['review'].apply(remove_special_characters)
# df['review'] = df['review'].apply(remove_stopwords)
df['review'].values[0]

'One of the other reviewers has mentioned that after watching just 1 Oz episode youll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wou

In [6]:
train_x, test_x, train_y, test_y = train_test_split(df['review'].values, df['sentiment'].values, test_size=0.2, random_state=22)
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape )

(40000,) (40000,) (10000,) (10000,)


In [7]:
vectorizer = CountVectorizer(max_features=150)
vectorizer = vectorizer.fit(train_x)

tfidf_vectorizer = TfidfVectorizer(max_features=500)
tfidf_vectorizer = tfidf_vectorizer.fit(train_x)



In [8]:
train_x_tfidf, test_x_tfidf = tfidf_vectorizer.transform(train_x), tfidf_vectorizer.transform(test_x)
train_x_bow, test_x_bow = vectorizer.transform(train_x), vectorizer.transform(test_x)

In [9]:
encoder = LabelBinarizer()
encoder = encoder.fit(train_y)

train_y, test_y = encoder.transform(train_y), encoder.transform(test_y)
print(train_y.shape, test_y.shape)

(40000, 1) (10000, 1)


In [10]:
lr_bw = LogisticRegression(penalty="l2", max_iter=500, C=1, random_state=42)
lr_bw = lr_bw.fit(train_x_bow, train_y)

lr_tfidf = LogisticRegression(penalty="l2", max_iter=500, C=1, random_state=42)
lr_tfidf = lr_tfidf.fit(train_x_tfidf, train_y)

In [11]:
train_pred = lr_bw.predict(train_x_bow)
train_acc_bow = accuracy_score(train_y, train_pred)
train_f1_bow = f1_score(train_y, train_pred)

test_pred = lr_bw.predict(test_x_bow)
test_acc_bow = accuracy_score(test_y, test_pred)
test_f1_bow = f1_score(test_y, test_pred)

train_pred = lr_tfidf.predict(train_x_tfidf)
train_acc_tfidf = accuracy_score(train_y, train_pred)
train_f1_tfidf = f1_score(train_y, train_pred)

test_pred = lr_tfidf.predict(test_x_tfidf)
test_acc_tfidf = accuracy_score(test_y, test_pred)
test_f1_tfidf = f1_score(test_y, test_pred)


In [15]:
pd.DataFrame({"train_acc" : [train_acc_bow*100, train_acc_tfidf*100],
              "train_f1" : [train_f1_bow*100, train_f1_tfidf*100], 
              "test_acc" : [test_acc_bow*100, test_acc_tfidf*100],
              "test_f1" :  [test_f1_bow*100, test_f1_tfidf*100]},
              index=['Bag of words', 'TFIDF'])


,train_acc,train_f1,test_acc,test_f1
Bag of words,75.8150,76.115939,75.38,75.786782
TFIDF,84.6825,84.840537,84.79,84.983710
